In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize

import utils.dev_config as dev_conf
import utils.preprocessing as prep
import utils.optimization as opt

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
i = 0

In [4]:
matrisome_df = prep.load_matrisome_df(matrisome_list)

In [5]:
seed = 123
rand = np.random.RandomState()

# Load and filter survival data

In [6]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["age_at_diagnosis", "race", "ethnicity"]
dep_cols = ["figo_stage"]
cat_cols = ["race", "ethnicity"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[i]}/survival_data.tsv", event_code)

In [7]:
filtered_survival_df = (
    prep.decode_figo_stage(survival_df[["sample_name"] + dep_cols + covariate_cols].dropna(), to="n")
        .pipe(pd.get_dummies, columns=cat_cols)
        .reset_index(drop = True)
        .pipe(prep.cols_to_front, ["sample_name", "figo_num"])
)
filtered_survival_df.columns = filtered_survival_df.columns.str.replace(' ', '_')
print(filtered_survival_df.shape)
# filtered_survival_df.head()

(255, 12)


# Load normalized matrisome count data

In [8]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[i]}/norm_matrisome_counts.tsv", sep='\t')
norm_filtered_matrisome_counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)], "geneID", "sample_name"
)
print(norm_filtered_matrisome_counts_t_df.shape)
# norm_filtered_matrisome_counts_t_df.head()

(255, 1009)


# Join survival and count data

In [9]:
joined_df = (
    pd.merge(filtered_survival_df, norm_filtered_matrisome_counts_t_df, on="sample_name")
        .set_index("sample_name")
)
print(joined_df.shape)
# joined_df.head()

(255, 1019)


# Optimize model

In [10]:
rand.seed(seed)
x_df, y_df = prep.shuffle_data(joined_df, rand)

In [11]:
def objective(h_params, X, y, r, verbose=True):
    if verbose:
        print(h_params)
    model = GradientBoostingClassifier(
        # 'exponential' seems to cause nan error
        loss="deviance",
        learning_rate=h_params[0],
        n_estimators=h_params[1],
        max_depth=h_params[2],
        max_features=h_params[3],
        random_state=r
    )
    # Standardize all variables (except one-hots)
    # Don't really need to do this for decision tree methods, but might as well
    # repeat the procedure that we're doing for the SVMs, just for posterity
    pipeline = make_pipeline(
        ColumnTransformer([
            ("standard", StandardScaler(), ["age_at_diagnosis"] + list(norm_filtered_matrisome_counts_t_df.columns[1:]))
        ], remainder="passthrough"),
        model
    )
    return -np.mean(cross_val_score(
        pipeline,
        X,
        y,
        cv=KFold(n_splits=5),
        n_jobs=-1,
        scoring="f1_weighted"
    ))

In [12]:
space = [
#     Categorical(["deviance", "exponential"], name="loss"),
    Real(1e-3, 1e-1, name="learning_rate"),
    Integer(int(1e2), int(1e3), name="n_estimators"),
    Integer(2, 5, name="max_depth"),
    Categorical(["auto", "sqrt", "log2"], name="max_features"),
]
n_initial = 10 * (len(space[3].categories))
n_calls = 100 * (len(space[3].categories))

In [13]:
res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df, rand),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls=n_calls,
    n_jobs=-1,
    callback=lambda x: opt.save_callback(x, "opt_gbc_h_params_TEST.tsv", sep="\t")
)

Iteration No: 1 started. Evaluating function at random point.
[0.00883574711308232, 873, 4, 'sqrt']
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 10.4799
Function value obtained: -0.4246
Current minimum: -0.4246
Iteration No: 2 started. Evaluating function at random point.
[0.013734488553428625, 174, 2, 'log2']
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.3328
Function value obtained: -0.3691
Current minimum: -0.4246
Iteration No: 3 started. Evaluating function at random point.
[0.043006379247018216, 606, 2, 'auto']
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 77.7841
Function value obtained: -0.4648
Current minimum: -0.4648
Iteration No: 4 started. Evaluating function at random point.
[0.08135279048011816, 521, 4, 'auto']
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 127.8417
Function value obtained: -0.4412
Current minimum: -0.4648
Iteration No: 5 started. Evaluating function at random point.


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 40 ended. Search finished for the next optimal point.
Time taken: 4.7852
Function value obtained: -0.4647
Current minimum: -0.4822
Iteration No: 41 started. Searching for the next optimal point.
[0.1, 1000, 2, 'log2']


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 4.6053
Function value obtained: -0.4821
Current minimum: -0.4822
Iteration No: 42 started. Searching for the next optimal point.
[0.1, 1000, 2, 'log2']


...

Iteration No: 298 ended. Search finished for the next optimal point.
Time taken: 16.2984
Function value obtained: -0.4644
Current minimum: -0.5073
Iteration No: 299 started. Searching for the next optimal point.
[0.1, 1000, 2, 'sqrt']


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 299 ended. Search finished for the next optimal point.
Time taken: 12.0352
Function value obtained: -0.4819
Current minimum: -0.5073
Iteration No: 300 started. Searching for the next optimal point.
[0.1, 1000, 2, 'sqrt']


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 300 ended. Search finished for the next optimal point.
Time taken: 17.4948
Function value obtained: -0.4766
Current minimum: -0.5073
